In [1]:
#install.packages("tidyverse")

In [2]:
library(tidyverse)
library(readxl)

── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.1     ✔ dplyr   1.0.5
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [3]:
url="https://cssbook.net/d/guns-polls.csv"
d = read_csv(url)
d = rename(d, rep=`Republican Support`, 
           dem=`Democratic Support`)
d = select(d, -URL)


d2 = filter(d, Question == "arm-teachers")
d2


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  Question = col_character(),
  Start = col_character(),
  End = col_character(),
  Pollster = col_character(),
  Population = col_character(),
  Support = col_double(),
  `Republican Support` = col_double(),
  `Democratic Support` = col_double(),
  URL = col_character()
)




Question,Start,End,Pollster,Population,Support,rep,dem
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
arm-teachers,2/23/18,2/25/18,YouGov/Huffpost,Registered Voters,41,69,20
arm-teachers,2/20/18,2/23/18,CBS News,Adults,44,68,20
arm-teachers,2/27/18,2/28/18,Rasmussen,Adults,43,71,24
arm-teachers,2/27/18,2/28/18,NPR/Ipsos,Adults,41,68,18
arm-teachers,3/3/18,3/5/18,Quinnipiac,Registered Voters,40,77,10
arm-teachers,2/26/18,2/28/18,SurveyMonkey,Registered Voters,43,80,11


In [4]:
# version of the guns polls with some errors
url="https://cssbook.net/d/guns-polls-dirty.csv"
d2 = read_csv(url) 

# Option 1: clean with direct assignment. 
# Note the need to specify d2$ everywhere

d2$rep2=str_replace_all(d2$rep, "[^0-9\\.]", "")
d2$rep2 = as.numeric(d2$rep2)
d2$Support2 = replace_na(d2$Support, 
                        mean(d2$Support, na.rm=T))

# Alternative, clean with mutate
# No need to specify d2$, 
# and we can assign to a new or existing object
cleaned = mutate(d2, 
    rep2 = str_replace_all(rep, "[^0-9\\.]", ""),
    rep2 = as.numeric(rep2),
    Support2 = replace_na(Support, 
        mean(Support, na.rm=TRUE)))

# Finally, you can create your own function

clean_num = function(x) {
    x = str_replace_all(x, "[^0-9\\.]", "")
    as.numeric(x)
}
cleaned = mutate(cleaned, rep3 = clean_num(rep))
head(cleaned)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  Question = col_character(),
  Start = col_character(),
  End = col_character(),
  Pollster = col_character(),
  Population = col_character(),
  Support = col_double(),
  rep = col_character(),
  dem = col_double()
)




Question,Start,End,Pollster,Population,Support,rep,dem,rep2,Support2,rep3
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
arm-teachers,2/23/18,2/25/18,YouGov/Huffpost,Registered Voters,41,69,20,69,41.0,69
arm-teachers,2/20/18,2/23/18,CBS News,Adults,NA,68,20,68,41.6,68
arm-teachers,2/27/18,2/28/18,Rasmussen,Adults,43,71d,24,71,43.0,71
arm-teachers,2/27/18,2/28/18,NPR/Ipsos,Adults,41,68,18,68,41.0,68
arm-teachers,3/3/18,3/5/18,Quinnipiac,Registered Voters,40,77,10,77,40.0,77
arm-teachers,2/26/18,2/28/18,SurveyMonkey,Registered Voters,43,80,11,80,43.0,80


In [5]:
groups = group_by(d, Question)
summarize(groups, m=mean(Support), sd=sd(Support))

Question,m,sd
<chr>,<dbl>,<dbl>
age-21,75.85714,6.011893
arm-teachers,42.00000,1.549193
background-checks,87.42857,7.322503
ban-assault-weapons,61.75000,6.440285
ban-high-capacity-magazines,67.28571,3.860669
mental-health-own-gun,85.83333,5.455884
repeal-2nd-amendment,10.00000,NA
stricter-gun-laws,66.45455,5.145165


In [6]:
d %>% group_by(Question) %>% 
  summarize(m=mean(Support), sd=sd(Support))

Question,m,sd
<chr>,<dbl>,<dbl>
age-21,75.85714,6.011893
arm-teachers,42.00000,1.549193
background-checks,87.42857,7.322503
ban-assault-weapons,61.75000,6.440285
ban-high-capacity-magazines,67.28571,3.860669
mental-health-own-gun,85.83333,5.455884
repeal-2nd-amendment,10.00000,NA
stricter-gun-laws,66.45455,5.145165


In [7]:
d = d %>% group_by(Question) %>% 
  mutate(mean = mean(Support), 
         deviation=Support - mean)
head(d)

Question,Start,End,Pollster,Population,Support,rep,dem,mean,deviation
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
age-21,2/20/18,2/23/18,CNN/SSRS,Registered Voters,72,61,86,75.85714,-3.857143
age-21,2/27/18,2/28/18,NPR/Ipsos,Adults,82,72,92,75.85714,6.142857
age-21,3/1/18,3/4/18,Rasmussen,Adults,67,59,76,75.85714,-8.857143
age-21,2/22/18,2/26/18,Harris Interactive,Registered Voters,84,77,92,75.85714,8.142857
age-21,3/3/18,3/5/18,Quinnipiac,Registered Voters,78,63,93,75.85714,2.142857
age-21,3/4/18,3/6/18,YouGov,Registered Voters,72,65,80,75.85714,-3.857143
